In [60]:
import geo_rendering
import pandas
import re
import cv2
import numpy as np

In [61]:
##update of the projection class from the âckage - used instead of geo_rendering.Projection

class Projection:

    def __init__(self, map_to_scale, scale_param=[1, 1], margin=[0, 0, 0, 0]):
        self.image_size = map_to_scale.image_size
        self.map_max_bound = map_to_scale.max_bound
        self.map_min_bound = map_to_scale.min_bound
        self.margin = margin  # (top, right, bottom, left) in pixels
        self.scale_param = scale_param  # x and y scaling parameters
        self.conversion = self.define_projection()[0]
        self.axis_to_center = self.define_projection()[1]

    def define_projection(self):
        """
            Calculates the conversion rate and axis on which to center the 
            converted coordinates
        """
        
        # We get the max 'coordinates' for both the target image and
        # the shape we want to draw
        image_x_max = self.image_size[0] - self.margin[1]
        image_y_max = self.image_size[1] - self.margin[2]
        image_x_min = 0 + self.margin[3]
        image_y_min = 0 + self.margin[0]
        map_x_max = self.map_max_bound[0] 
        map_y_max = self.map_max_bound[1]
        map_x_min = self.map_min_bound[0]
        map_y_min = self.map_min_bound[1]

        # we check which size is bigger to know based on which axis we want
        # to scale our shape to
        # we do the comparison using the aspect ratio expectations (dividing
        # each axis by the size of the target axis in the new scale)
        ratio_x = (map_x_max - map_x_min)/(image_x_max - image_x_min)
        ratio_y = (map_y_max - map_y_min)/(image_y_max - image_y_min)
        if ratio_x > ratio_y:
            conversion = 1 / ratio_x
            axis_to_center = 'y'  # we store the axis we will want to center on
            # based on which axis we perform the scaling from
        else:
            conversion = 1 / ratio_y
            axis_to_center = 'x'

        return conversion, axis_to_center

    def apply_projection(self, coords, inverse=False):
        """
            applies the conversion on coordinates ;
            the inverse argument allows to go from one coordinate system
            (the original one), to the new one
        """
        x = coords[0]
        y = coords[1]
        x_min = self.map_min_bound[0]
        y_min = self.map_min_bound[1]

        if inverse is False:
            # to be able to center the image, we first translate the
            # coordinates to the origin
            # we apply the scaling ratio to each axis
            x = (x - x_min) * self.conversion * self.scale_param[0]
            y = (y - y_min) * self.conversion * self.scale_param[1]
        else:
            x = x / self.conversion + x_min
            y = y / self.conversion + y_min

        coords = [x, y]
        return coords

    def apply_translation(self, coords):
        """
            Translates the coordinates along the axis to center in order to center the map
        """
        axis_to_center = self.axis_to_center
        image_x_max = self.image_size[0] - self.margin[1]
        image_y_max = self.image_size[1] - self.margin[2]
        image_x_min = 0 + self.margin[3]
        image_y_min = 0 + self.margin[0]
        map_x_max = self.map_max_bound[0]
        map_y_max = self.map_max_bound[1]
        map_max_converted = self.apply_projection((map_x_max, map_y_max))
        map_x_min = self.map_min_bound[0]
        map_y_min = self.map_min_bound[1]
        map_min_converted = self.apply_projection((map_x_min, map_y_min))

        if axis_to_center == 'x':
            map_x_max_conv = map_max_converted[0]
            map_x_min_conv = map_min_converted[0]
            center_translation = ((image_x_max - image_x_min)
                                  - (map_x_max_conv - map_x_min_conv))/2
        else:
            map_y_max_conv = map_max_converted[1]
            map_y_min_conv = map_min_converted[1]
            center_translation = ((image_y_max - image_y_min)
                                  - (map_y_max_conv - map_y_min_conv))/2

        # we center the map on the axis that was not used to scale the image
        if axis_to_center == 'x':
            coords[0] = coords[0] + center_translation
        else:
            coords[1] = coords[1] + center_translation

        # we mirror the image to match the axis alignment
        coords[1] = image_y_max - coords[1]

        return coords

In [62]:
shp_path = "/Users/acoullandreau/Desktop/carto_cda_lr/communes/communes-20190101.shp"

In [63]:
base_shapefile = geo_rendering.ShapeFile(shp_path)

In [64]:
df_charente = base_shapefile.df_sf[base_shapefile.df_sf['insee'].str.contains(r'^17*')]
df_charente.head()

,insee,nom,wikipedia,surf_ha,coords
212,17060,Boutenac-Touvent,fr:Boutenac-Touvent,318.0,"[(-0.7783741999999998, 45.49204320041383), (-0..."
213,17133,Cravans,fr:Cravans,1477.0,"[(-0.7421043999999999, 45.5993678004), (-0.742..."
214,17244,Montpellier-de-Médillan,fr:Montpellier-de-Médillan,1500.0,"[(-0.7833256999999998, 45.63373440039544), (-0..."
221,17417,Salignac-de-Mirambeau,fr:Salignac-de-Mirambeau,760.0,"[(-0.5028819999999999, 45.344952100431634), (-..."
222,17305,Rouffignac,fr:Rouffignac,1476.0,"[(-0.4771612999999999, 45.32736130043369), (-0..."


In [65]:
cda = [17010, 17028, 17059, 17094, 17109, 17136, 17142, 17153, 17190, 17193, 17194, 17300, 17200, 17222,
       17245, 17264, 17274, 17291, 17315, 17373, 17391, 17413, 17414, 17407, 17420, 17443, 17466, 17483,
      17003, 17009, 17032, 17447, 17208, 17376, 17472, 17007, 17008, 17057, 17065, 17091, 17107, 17166,
      17168, 17186, 17267, 17353, 17396, 17463, 17480]
cda = [str(i) for i in cda]

df_cda = base_shapefile.df_sf[base_shapefile.df_sf['insee'].isin(cda)]

In [66]:
#base_shapefile.df_sf=df_charente

In [67]:
base_map = geo_rendering.Map(base_shapefile, [1920, 1080], [235,206,135])

In [68]:
zoom_shapefile = geo_rendering.ShapeFile(shp_path)

In [69]:
zoom_shapefile.build_shape_dict(df_cda)

{303: <geo_rendering.shape_class.ShapeOnMap at 0x702c8f6d8>,
 19533: <geo_rendering.shape_class.ShapeOnMap at 0x7043b55f8>,
 19782: <geo_rendering.shape_class.ShapeOnMap at 0x7043b5e80>,
 29113: <geo_rendering.shape_class.ShapeOnMap at 0x2dbba3c18>,
 29114: <geo_rendering.shape_class.ShapeOnMap at 0x351c55f28>,
 29115: <geo_rendering.shape_class.ShapeOnMap at 0x2db6dfe80>,
 32126: <geo_rendering.shape_class.ShapeOnMap at 0x2db6df780>,
 32131: <geo_rendering.shape_class.ShapeOnMap at 0x2db6dfd30>,
 32152: <geo_rendering.shape_class.ShapeOnMap at 0x2db6df7b8>,
 32386: <geo_rendering.shape_class.ShapeOnMap at 0x351c55cf8>,
 32413: <geo_rendering.shape_class.ShapeOnMap at 0x351c555f8>,
 32421: <geo_rendering.shape_class.ShapeOnMap at 0x351c55860>,
 32423: <geo_rendering.shape_class.ShapeOnMap at 0x351c55c88>,
 32424: <geo_rendering.shape_class.ShapeOnMap at 0x351c55f60>,
 32430: <geo_rendering.shape_class.ShapeOnMap at 0x351c554a8>,
 32437: <geo_rendering.shape_class.ShapeOnMap at 0x351c55

In [70]:
zoom_shapefile.df_sf = df_cda

In [71]:
zoom_map = geo_rendering.Map(zoom_shapefile, [1920, 1080], [235,206,135])

In [72]:
zoom_projection = Projection(zoom_map, scale_param=[0.65, 1])
base_map.projection = zoom_projection

In [73]:
for city_id in base_map.shape_dict_filt:
    shape = base_map.shape_dict_filt[city_id]
    shape.project_shape_coords(base_map.projection)

for city_id in base_map.shape_dict:
    shape = base_map.shape_dict[city_id]
    shape.project_shape_coords(base_map.projection)

In [74]:
zoom_shapefile.df_sf

,insee,nom,wikipedia,surf_ha,coords
303,17109,Clavette,fr:Clavette (Charente-Maritime),630.0,"[(-1.0616862999999999, 46.13546370032097), (-1..."
19533,17166,Forges,fr:Forges (Charente-Maritime),1370.0,"[(-0.9199342999999998, 46.110315700325046), (-..."
19782,17373,Saint-Médard-d'Aunis,fr:Saint-Médard-d'Aunis,2270.0,"[(-1.0093919999999996, 46.137786700320596), (-..."
29113,17193,La Jarne,fr:La Jarne,842.0,"[(-1.1040996999999997, 46.12057230032338), (-1..."
29114,17420,Salles-sur-Mer,fr:Salles-sur-Mer,1412.0,"[(-1.0890913999999996, 46.09409380032765), (-1..."
29115,17413,Saint-Vivien,fr:Saint-Vivien (Charente-Maritime),837.0,"[(-1.0790503999999996, 46.080813700329784), (-..."
32126,17483,Yves,fr:Yves (Charente-Maritime),2602.0,"[(-1.0839906999999998, 46.053889900334056), (-..."
32131,17353,Saint-Laurent-de-la-Prée,fr:Saint-Laurent-de-la-Prée,2617.0,"[(-1.0655682999999998, 45.96733900034753), (-1..."
32152,17463,Vergeroux,fr:Vergeroux,560.0,"[(-1.0044081999999999, 45.97719990034603), (-1..."
32386,17267,Nuaillé-d'Aunis,fr:Nuaillé-d'Aunis,1649.0,"[(-0.9640837999999999, 46.25192760030168), (-0..."


In [75]:
color_zone_1 = [138, 138, 243]
color_zone_2 = [107, 137, 255]
color_zone_3 = [181, 202, 160]
color_zone_4 = [202,233,205]
color_zone_5 = [208,237,250]
color_dict = {303:color_zone_3, 19782:color_zone_4, 29113:color_zone_2, 29114:color_zone_3, 
              29115:color_zone_3, 32126:color_zone_4,  32437:color_zone_4, 32802:color_zone_3,
              32874:color_zone_3, 33300:color_zone_2, 33301:color_zone_2, 33303:color_zone_1,
              33304:color_zone_2, 33305:color_zone_2, 33309:color_zone_3, 33310:color_zone_3,
              33311:color_zone_3, 33314:color_zone_3, 33315:color_zone_4, 33316:color_zone_2,
              33317:color_zone_3, 33320:color_zone_1, 33321:color_zone_4, 33325:color_zone_2,
              33326:color_zone_3, 33331:color_zone_3, 33347:color_zone_4, 33353:color_zone_2,
              33252:color_zone_4, 32423:color_zone_4, 33299:color_zone_4, 33253:color_zone_4,
              32442:color_zone_4, 32413:color_zone_4, 33297:color_zone_4, 33185:color_zone_5,
              32488:color_zone_5, 33298:color_zone_5, 33367:color_zone_5, 32430:color_zone_5,
              32804:color_zone_5, 19533:color_zone_5, 33360:color_zone_5, 32424:color_zone_5,
              32386:color_zone_5, 32131:color_zone_5, 32421:color_zone_5,
              32152:color_zone_5, 33355:color_zone_5}

name_dict = {303:'Clavette', 19782:"Saint-Medard-d'Aunis", 29113:"La Jarne", 29114:"Salles-sur-Mer",
             29115:"Saint-Vivien",  32126:"Yves",  32437:"Verines",  32802:"Bourgneuf",  32874:"La Jarrie",
             33300:"Saint-Rogatien", 33301:"Dompierre-sur-Mer", 33303:"Aytre", 33304:"Lagord",
             33305:"Puilboreau", 33309:"Nieul-sur-Mer", 33310:"Marsilly", 33311:"Sainte-Soulle",
             33314:"Esnandes", 33315:"Saint-Christophe", 33316:"Angoulins", 33317:"Chatelaillon-Plage",
             33320:"La Rochelle", 33321:"Thaire d'Aunis", 33325:"Perigny", 33326:"Montroy", 33331:"Saint-Xandre",
             33347:"Croix-Chapeau", 33353:"L'Houmeau", 33252:"Aigrefeuille", 32423:"Angliers",
             33299:"Ballon", 33253:"Le Thou", 32442:"Longeves", 32413:"Saint-Ouen d'Aunis",
            33297:"Villedoux", 33185:"Anais", 32488:"Andilly", 33298:"Bouhet", 33367:"Breuil-Magne",
            32430:"Charron", 32804:"Cire d'Aunis", 19533:"Forges", 33360:"Fouras", 32424:"Le Gue d'Allere",
            32386:"Nuaille d'Aunis", 32131:"Saint-Laurent de la Pre", 32421:"Saint-Sauveur d'Aunis",
            32152:"Vergeroux", 33355:"Virson"}

accents_manquants = [19782, 32437, 33303, 33317, 33321, 33325, 32442, 33367, 32804, 32424, 32386, 32131]
                                                                      

In [76]:
for city_id in base_map.shape_dict_filt:
    shape = base_map.shape_dict_filt[city_id]
    if city_id in zoom_map.shape_dict:
        shape.color_fill = color_dict[city_id]
    elif city_id in df_charente.index:
        shape.color_fill =  [240, 254, 254]
        shape.color_line = [0, 0, 0]
    else:
        shape.color_fill = [255, 255, 255]
        shape.color_line = [0, 0, 0]

In [77]:
base_map.render_map()

In [78]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [79]:
for city_id in base_map.shape_dict_filt:
    shape = base_map.shape_dict_filt[city_id]
    base_map.shape_dict_filt[city_id].fill_in_shape(base_map.map_file)
    if city_id in zoom_map.shape_dict:
        pts = np.array(shape.points, np.int32)
        cv2.polylines(base_map.map_file, [pts], True, (0, 0, 0), 1, cv2.LINE_AA)
    else:
        pts = np.array(shape.points, np.int32)
        cv2.polylines(base_map.map_file, [pts], True, (0, 0, 0), 1, cv2.LINE_AA)
    
    if city_id in df_cda.index:
        city_name = name_dict[city_id]
        x = int(shape.center[0])-25
        y = int(shape.center[1])+5
        city_center = (x, y)
        cv2.putText(base_map.map_file, '{}'.format(city_name), city_center, font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
cv2.putText(base_map.map_file, 'Legende', (30,850), font, 0.9, (0, 0, 0), 1, cv2.LINE_AA)
cv2.rectangle(base_map.map_file,(30,880),(70,910),color_zone_1, -1)
cv2.putText(base_map.map_file, 'Zone 1', (100, 900), font, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
cv2.rectangle(base_map.map_file,(30,910),(70,950),color_zone_2, -1)
cv2.putText(base_map.map_file, 'Zone 2', (100, 940), font, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
cv2.rectangle(base_map.map_file,(30,950),(70,990),color_zone_3, -1)
cv2.putText(base_map.map_file, 'Zone 3', (100, 980), font, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
cv2.rectangle(base_map.map_file,(30,990),(70,1030),color_zone_4, -1)
cv2.putText(base_map.map_file, 'Zone 4', (100, 1020), font, 0.7, (0, 0, 0), 1, cv2.LINE_AA)
cv2.rectangle(base_map.map_file,(30,1030),(70,1070),color_zone_5, -1)
cv2.putText(base_map.map_file, 'Zone 5', (100, 1060), font, 0.7, (0, 0, 0), 1, cv2.LINE_AA)



array([[[235, 206, 135],
        [235, 206, 135],
        [235, 206, 135],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[235, 206, 135],
        [235, 206, 135],
        [235, 206, 135],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[235, 206, 135],
        [235, 206, 135],
        [235, 206, 135],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[235, 206, 135],
        [235, 206, 135],
        [235, 206, 135],
        ...,
        [240, 254, 254],
        [240, 254, 254],
        [240, 254, 254]],

       [[235, 206, 135],
        [235, 206, 135],
        [235, 206, 135],
        ...,
        [240, 254, 254],
        [240, 254, 254],
        [240, 254, 254]],

       [[235, 206, 135],
        [235, 206, 135],
        [235, 206, 135],
        ...,
        [240, 254, 254],
        [240, 254, 254],
        [240, 254, 254]]

In [80]:
cv2.imwrite( "output_map/Carte_CDA.jpg", base_map.map_file)

True

In [30]:
cv2.imshow('image', base_map.map_file)
cv2.waitKey(0)
cv2.destroyAllWindows()